In [1]:
import pandas as pd

from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

### Q1

In [2]:
jan_trips = pd.read_parquet("data/yellow_tripdata_2023-01.parquet")

jan_trips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

### Q2

In [3]:
jan_trips["duration"] = jan_trips["tpep_dropoff_datetime"] - jan_trips["tpep_pickup_datetime"]
jan_trips["duration"] = jan_trips["duration"].apply(lambda x: x.total_seconds() / 60)

jan_trips["duration"].describe()

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
25%      7.116667e+00
50%      1.151667e+01
75%      1.830000e+01
max      1.002918e+04
Name: duration, dtype: float64

### Q3

In [4]:
processed_jan_trips = jan_trips[(jan_trips["duration"] >= 1) & (jan_trips["duration"] <= 60)]

processed_jan_trips.shape[0] / jan_trips.shape[0]

0.9812202822125979

### Q4

In [5]:
encoder = OneHotEncoder(handle_unknown="ignore")

train_locations = processed_jan_trips[["PULocationID", "DOLocationID"]].astype(str)
encoder.fit_transform(train_locations)

<3009173x515 sparse matrix of type '<class 'numpy.float64'>'
	with 6018346 stored elements in Compressed Sparse Row format>

### Q5

In [6]:
X_train = encoder.transform(train_locations)
y_train = processed_jan_trips["duration"]

model = LinearRegression()
model.fit(X_train, y_train)

mean_squared_error(y_train, model.predict(X_train), squared=False)

7.649261027964222

### Q6

In [7]:
feb_trips = pd.read_parquet("data/yellow_tripdata_2023-02.parquet")
feb_trips["duration"] = feb_trips["tpep_dropoff_datetime"] - feb_trips["tpep_pickup_datetime"]
feb_trips["duration"] = feb_trips["duration"].apply(lambda x: x.total_seconds() / 60)

processed_feb_trips = feb_trips[(feb_trips["duration"] >= 1) & (feb_trips["duration"] <= 60)]

val_locations = processed_feb_trips[["PULocationID", "DOLocationID"]].astype(str)

X_val = encoder.transform(val_locations)
y_val = processed_feb_trips["duration"]

mean_squared_error(y_val, model.predict(X_val), squared=False)

7.811832455157901